# to do：
- 回顾之前写的划分MFD的notebook，添加定量分析的函数（CHindex（在写好的部分论文里），NSk，TV）。
- 选出数据，喂到这个notebook里
- adaptation loss前面的lambda用两个类之间的KL散度
- boundary adjustment的时候考虑双向，尽量提供错位的数据，即两个方向可能属于不同类

- 寻找reinforcement learning用于CAV的算法

In [1]:
import time
import math
import tensorflow as tf
import keras 
import numpy as np
import pandas as pd
import ipdb
from keras.models import load_model,Model
from keras.engine.topology import Layer

ModuleNotFoundError: No module named 'keras.models'

In [ ]:
# 定义融合层，将深度学习算法与历史均值算法融合
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [2]:
#定义精度评价指标。为防止0值附近相对误差过大而导致的异常，定义mask层。
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

## 把速度矩阵分类

In [87]:
randseed = 3
class1 = 0
class2 = 1
v = pd.read_csv('./data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('./res/%i_id_402_withclass.csv'%randseed, index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

v_class1 = v[v['class_i']==class1]
v_class2 = v[v['class_i']==class2]

## 制作 nearest_road_id.csv 和speed.csv

In [7]:
pd.read_csv('./data/id2000.csv', index_col=0)

,id2,lat,long,id,num_road,class
0,2001,116.42769,39.94485,HI7000d,4,0
1,2002,116.42764,39.94330,HI7001a,3,0
2,2003,116.42782,39.93705,HI7002a,2,0
3,2005,116.42832,39.92993,HI7039a,3,0
4,2006,116.42785,39.92513,HI7003a,3,0
...,...,...,...,...,...,...
202,61004,116.29400,39.97990,HI9037g,3,3
203,61005,116.29300,39.98990,HI9097c,3,3
204,61006,116.29000,39.99350,HI9098c,3,3
205,61007,116.28600,39.99870,HI9099c,3,3


In [8]:
dist_mat = pd.read_csv('./data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('./data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

dist_mat

id2,2001,2002,2003,2005,2006,2007,2009,2010,2012,2016,...,51009,51010,61001,61002,61003,61004,61005,61006,61007,61009
id2,,,,,,,,,,,,,,,,,,,,,
2001,0,292,1324,2037,2690,3061,4542,4334,5957,7266,...,8499,8519,13379,14289,17292,16678,17434,17513,19099,19080
2002,420,0,1103,1816,2469,2840,4266,4113,5681,6990,...,8762,8782,13593,14503,17555,16941,17163,17145,18731,18712
2003,1286,1315,0,1161,1814,2185,3746,3458,5161,6470,...,9302,9322,13913,14823,17358,16744,17483,17465,19051,19032
2005,3149,3005,2008,0,1188,1983,2893,3397,4317,5626,...,11214,11234,15688,16598,18657,18043,18799,19488,21074,21055
2006,2463,2492,1589,1056,0,1147,2441,2420,4384,5693,...,10353,10373,14920,15830,17889,17275,18031,18720,20306,20287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61004,15651,15797,15964,16834,17460,17808,19149,19397,21047,21759,...,7845,8686,2929,2662,774,0,1238,1880,4537,4518
61005,15994,16140,16307,17177,17803,18151,19492,19740,21390,22102,...,8383,9224,3274,3007,4690,4076,0,908,3565,3546
61006,16443,16589,16756,17626,18252,18600,19941,20189,21839,22551,...,8832,9673,3723,3456,5331,4717,907,0,3069,3050


In [10]:
near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])
near_id

id2,2001,2002,2003,2005,2006,2007,2009,2010,2012,2016,...,51009,51010,61001,61002,61003,61004,61005,61006,61007,61009
id2,,,,,,,,,,,,,,,,,,,,,
2001,0,1,48,2,47,3,4,102,5,104,...,130,132,124,131,126,125,128,127,123,129
2002,1,0,2,48,3,47,4,104,5,46,...,130,132,124,131,126,125,128,127,123,129
2003,2,3,0,1,4,48,5,47,50,104,...,132,168,131,124,126,125,128,127,123,129
2005,3,4,5,2,6,50,1,0,7,53,...,124,131,206,205,126,125,128,127,123,129
2006,4,3,5,2,7,6,0,1,48,53,...,131,204,126,125,206,205,128,127,123,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61004,202,145,201,203,204,144,200,199,88,146,...,105,114,108,107,106,110,109,113,111,112
61005,203,204,145,144,200,199,206,205,146,88,...,105,114,108,107,106,110,109,113,111,112
61006,204,203,145,206,205,144,200,199,146,88,...,105,114,108,107,106,110,109,113,111,112


## todo
- 设计代码：选出n条路（比如30条），然后找出对应的near_rd矩阵和v矩阵
- 思考node和detector的问题

## 以上做好了near_road矩阵，接下来做flow/speed矩阵

In [31]:
seg = pd.read_csv('./data/segement.csv', header=None)
seg

,0,1,2,3,4,5,6,7
0,2001,东直门桥北728米（内环）,116.42769,39.94485,4,3,HI7000d,HI7060b
1,2002,东直门桥北539米人行天桥西（外环）,116.42764,39.94330,3,3,HI7001a,HI7000c
2,2003,东直门南机非隔离带内现况视频旁（外环）,116.42782,39.93705,2,3,HI7002a,HI7001c
3,2004,东四十条桥北绿地内中航大厦南20米,116.42760,39.93413,3,3,NaN,NaN
4,2005,东四十条桥南富华大厦附近（外环）,116.42832,39.92993,3,3,HI7039a,HI7002c
...,...,...,...,...,...,...,...,...
227,61005,万泉河桥北第一天桥北,116.29300,39.98990,3,3,HI9097c,HI9037a
228,61006,万泉河桥北第二天桥北,116.29000,39.99350,3,3,HI9098c,HI9097a
229,61007,万泉河桥北无名北立交桥,116.28600,39.99870,3,3,HI9099c,HI9098a
230,61008,无名北立交桥桥北,116.28200,40.00340,3,3,HI9100c,HI9099a


In [34]:
def get_node(det, seg):
    # det is one single detector id
    # node is one single node id
    
    # seg = pd.read_csv('./data/segement.csv', header=None)
    try:
        node_info = seg[seg[6]==det]
        node = node_info.iloc[0, 0]
    except:
        node_info = seg[seg[7]==det]
        node = node_info.iloc[0, 0]
        
    return node

In [103]:
def get_class_with_node(seg, v_class):
    det_list_class = np.array([])
    try:
        v_class.insert(1, 'id2', '')  # id2 mean node id
    except:
        v_class['id2'] = ''
        
    for i in range(len(v_class)):
        det_list_class = np.append(det_list_class, v_class.iloc[i, 0])
        v_class.iloc[i, 1] = get_node(v_class.iloc[i, 0], seg)
    
    return det_list_class, v_class

In [107]:
seg = pd.read_csv('./data/segement.csv', header=None)

det_list_class1, v_class1 = get_class_with_node(seg, v_class1)

C:\Users\10169\.conda\envs\dan_traff\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [109]:
def rds_mat(old_dist_mat, det_ids):
    # get a matrix that contains n raods that have specified node id s
    node_ids = np.array([])
    for i in det_ids:
        node_ids = np.append(node_ids, get_node(i, seg))
        
    new_dist_mat = old_dist_mat.loc[node_ids, node_ids]
    old_dist_mat = np.array(old_dist_mat)
    new_near_id_mat = np.argsort(new_dist_mat)
    return new_near_id_mat

In [112]:
near_road = rds_mat(dist_mat, det_list_class1[:30])

In [113]:
v_class1[v_class1['id'].isin(det_list_class1[:30])]

,id,id2,2015-07-01 00:10:00,2015-07-01 00:20:00,2015-07-01 00:30:00,2015-07-01 00:40:00,2015-07-01 00:50:00,2015-07-01 01:00:00,2015-07-01 01:10:00,2015-07-01 01:20:00,...,2015-07-31 22:40:00,2015-07-31 22:50:00,2015-07-31 23:00:00,2015-07-31 23:10:00,2015-07-31 23:20:00,2015-07-31 23:30:00,2015-07-31 23:40:00,2015-07-31 23:50:00,2015-08-01 00:00:00,class_i
19,HI3002b,2058,50.980000,54.880000,52.560000,50.560000,52.580000,53.800000,48.420000,47.020000,...,58.006667,56.425333,55.510345,56.842667,54.960000,53.280000,57.757143,55.920000,56.700000,0
20,HI3002d,2057,77.460000,82.340000,76.420000,80.124000,82.520000,79.121429,80.260000,79.760000,...,70.000000,67.540000,69.800000,68.060000,73.800000,82.400000,79.800000,72.580000,75.220000,0
21,HI3008b,2066,70.060000,64.840000,79.727407,84.577037,83.404828,85.880000,80.209524,82.420000,...,70.060000,73.420000,76.274545,68.560000,67.180000,67.226667,64.660000,65.020000,70.296800,0
22,HI3008d,2064,84.832123,87.976000,90.000000,87.930000,90.000000,90.000000,89.500000,76.953913,...,68.500000,68.500000,68.568889,78.200000,78.200000,78.200000,78.563200,78.200000,78.200000,0
23,HI3009b,2067,66.100000,68.140000,67.740000,67.380000,69.243448,67.493333,67.480000,67.100000,...,24.800000,36.155556,45.789655,52.927407,53.540000,50.220000,49.500000,50.087143,38.580000,0
24,HI3009d,2066,90.000000,90.000000,89.780000,89.822222,89.826897,90.000000,89.820000,90.000000,...,83.680000,89.400000,89.204545,87.200000,89.520000,89.577778,89.500000,87.276667,86.889600,0
40,HI7000c,2002,56.300000,57.480000,56.980000,56.640000,60.680000,56.720000,58.300000,60.160000,...,24.680000,24.920000,23.280000,32.480000,46.740000,46.680000,45.980000,49.220000,52.880000,0
41,HI7000d,2001,81.000000,81.680000,83.920000,81.900000,70.740000,61.900000,60.500000,67.000000,...,64.500000,52.600000,72.100000,72.840000,73.100000,76.175000,64.000000,82.180000,79.780000,0
42,HI7001a,2002,33.460000,34.820000,34.580000,33.920000,36.540000,36.620000,33.940000,32.480000,...,16.520000,24.640000,35.680000,33.040000,34.600000,35.260000,35.060000,36.120000,35.580000,0
43,HI7001c,2003,90.000000,90.000000,89.228571,90.000000,90.000000,90.000000,89.216800,89.293077,...,39.346667,58.278519,60.080000,66.490000,59.920000,64.308696,64.854167,64.660000,68.562500,0


# 源代码如下

In [8]:
near_road = np.array(pd.read_csv('./data/network/2small_network_nearest_road_id.csv',header = 0))
flow = np.array(pd.read_csv('./data/network/2small_network_speed.csv', header= 0)) #注意header=0 or None

# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 29 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])
        

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后10%数据为测试集
image_train_source = image3[:np.shape(image3)[0]*1//10]
image_test_source = image3[np.shape(image3)[0]*1//10:]
label_train_source = label[:np.shape(label)[0]*1//10]
label_test_source = label[np.shape(label)[0]*1//10:]

day_train_source = day[:np.shape(day)[0]*1//10]
day_test_source = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(4620, 5, 12, 30)
(4620, 30, 3)
(4620, 30, 3)
input done 0.0458782


In [9]:
near_road = np.array(pd.read_csv('./data/transfer_learning_traffic_data/small_network_nearest_road_id.csv',header = 0))
flow = np.array(pd.read_csv('./data/transfer_learning_traffic_data/small_network_speed.csv', header= 0)) #注意header=0 or None
# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

time3 = time.time()

k = 5 # 参数k为需考虑的最近路段数
t_p = 29 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(5min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(5min颗粒度)
num_links = 30 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train_target = image3[:np.shape(image3)[0]*1//10]
image_test_target = image3[np.shape(image3)[0]*1//10:]
label_train_target = label[:np.shape(label)[0]*1//10]
label_test_target = label[np.shape(label)[0]*1//10:]

day_train_target = day[:np.shape(day)[0]*1//10]
day_test_target = day[np.shape(day)[0]*1//10:]


time4 = time.time()
print('input done %g' % (time4-time3))

(4620, 5, 12, 30)
(4620, 30, 3)
(4620, 30, 3)
input done 0.0524616


In [10]:
#模型构建
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = num_links,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = num_links,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_4[0][0]      
__________________________________________________________________________________________________
average_pooling2d_3 (AveragePoo (None, 5, 12, 30)    0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [11]:
#参数加载
finish_model.load_weights('./model/source.h5')


In [31]:
#模型预测
model_pre = finish_model.predict([image_test_target,day_test_target])


In [32]:
#预测结果存储
# model_pre = np.reshape(model_pre,[103, 6])
# model_pre1 = pd.DataFrame(model_pre)
# model_pre1.to_csv('预测值.csv', index = False)

In [33]:
#transfer without FT 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_source)
smape_mean = smape_loss_func(model_pre, label_test_source)
mae_mean = mae_loss_func(model_pre, label_test_source)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.410740088181874
smape = 0.31877275140460637
mae = 7.512052742950628


In [34]:

middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_2').output)

In [35]:
middle_result_source = middle.predict([image_train_source, day_train_source])
middle_result_target = middle.predict([image_train_target, day_train_target])

In [36]:
from sklearn import metrics
def mmd (x, y):
    return metrics.mean_squared_error(x,y)

In [37]:
mmd (middle_result_source, middle_result_target)

1848.1279

In [38]:
from keras import backend as K

In [39]:
loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
overall_loss = loss1 + loss2

In [40]:
def new_loss(output_final, label_train_target):
    middle = Model(inputs=[input_data, input_HA],outputs=finish_model.get_layer('dense_2').output)
    middle_result_source = middle.predict([image_train_source, day_train_source])
    middle_result_target = middle.predict([image_train_target, day_train_target])

    loss1 = K.mean(K.square(output_final - label_train_target), axis=-1) 
    loss2 = 0.001 * mmd (middle_result_source, middle_result_target)
    overall_loss = loss1 + loss2
    return overall_loss


In [41]:
finish_model.compile(optimizer='adam',loss=new_loss)

In [42]:
finish_model.fit([image_train_target, day_train_target], label_train_target, epochs=100, batch_size=462,
validation_data=([image_test_target,day_test_target], label_test_target))

Train on 462 samples, validate on 4158 samples
Epoch 1/100
462/462 [==============================] - 1s 2ms/step - loss: 40.2202 - val_loss: 45.2030
Epoch 2/100
462/462 [==============================] - 0s 248us/step - loss: 40.7406 - val_loss: 43.0695
Epoch 3/100
462/462 [==============================] - 0s 248us/step - loss: 39.8947 - val_loss: 42.2472
Epoch 4/100
462/462 [==============================] - 0s 238us/step - loss: 40.1046 - val_loss: 41.8395
Epoch 5/100
462/462 [==============================] - 0s 244us/step - loss: 40.1671 - val_loss: 41.6362
Epoch 6/100
462/462 [==============================] - 0s 231us/step - loss: 39.8075 - val_loss: 41.6065
Epoch 7/100
462/462 [==============================] - 0s 237us/step - loss: 39.5556 - val_loss: 41.7633
Epoch 8/100
462/462 [==============================] - 0s 240us/step - loss: 39.4774 - val_loss: 42.0697
Epoch 9/100
462/462 [==============================] - 0s 240us/step - loss: 39.2657 - val_loss: 42.2636
Epoch 10/1

Epoch 78/100
462/462 [==============================] - 0s 227us/step - loss: 26.2033 - val_loss: 40.4117
Epoch 79/100
462/462 [==============================] - 0s 233us/step - loss: 25.9682 - val_loss: 40.4637
Epoch 80/100
462/462 [==============================] - 0s 237us/step - loss: 25.9601 - val_loss: 40.4940
Epoch 81/100
462/462 [==============================] - 0s 247us/step - loss: 25.8471 - val_loss: 40.5304
Epoch 82/100
462/462 [==============================] - 0s 231us/step - loss: 25.6674 - val_loss: 40.6440
Epoch 83/100
462/462 [==============================] - 0s 231us/step - loss: 25.6055 - val_loss: 40.7078
Epoch 84/100
462/462 [==============================] - 0s 237us/step - loss: 25.6789 - val_loss: 40.8083
Epoch 85/100
462/462 [==============================] - 0s 229us/step - loss: 25.5219 - val_loss: 40.9045
Epoch 86/100
462/462 [==============================] - 0s 224us/step - loss: 25.5746 - val_loss: 40.8996
Epoch 87/100
462/462 [========================

In [43]:
model_pre = finish_model.predict([image_test_target,day_test_target])

In [44]:
#transfer with DAN 预测精度计算

mape_mean = mape_loss_func(model_pre, label_test_target)
smape_mean = smape_loss_func(model_pre, label_test_target)
mae_mean = mae_loss_func(model_pre, label_test_target)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.21938529190138537
smape = 0.18145137157824318
mae = 4.557769227320512


In [46]:
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test_target[:,i,:])
    mape_list.append(a1)

mape_pd = pd.Series(mape_list)
mape_pd.sort_values()

25    0.084843
0     0.094769
7     0.099638
1     0.103315
8     0.119802
17    0.121618
20    0.132502
6     0.137082
3     0.145460
14    0.146810
15    0.147304
5     0.153893
12    0.161637
2     0.175947
21    0.184446
10    0.194028
19    0.206151
28    0.223165
11    0.234875
9     0.238887
18    0.261291
13    0.311152
27    0.317819
24    0.337854
29    0.341141
23    0.360701
26    0.364405
4     0.370063
16    0.386819
22    0.424432
dtype: float64